<a href="https://colab.research.google.com/github/harishtlv/Emotion_Detection/blob/main/Distillation_test1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%bash
git clone https://github.com/openvinotoolkit/anomalib.git
cd anomalib
pip install -e .

In [ ]:
# !pip install anomalib
!pip install pytorch-lightning
!pip install lightning
!pip install kornia
!pip install timm
!pip install FrEIA
!pip install open-clip-torch

In [8]:
from anomalib.data import MVTec
datamodule = MVTec()

In [28]:
# !pip install anomalib
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models
from torch.utils.data import DataLoader
import pytorch_lightning as pl
from torchvision.transforms import Compose, Resize, ToTensor, Normalize
from torchvision.datasets import ImageFolder

from anomalib.models import Patchcore

class StudentModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = models.resnet18(pretrained=True)
        num_ftrs = self.model.fc.in_features
        self.model.fc = nn.Linear(num_ftrs, 1000)  # Adjust output size to match teacher

    def forward(self, x):
        return self.model(x)

class DistillationModel(pl.LightningModule):
    def __init__(self, teacher_model, student_model, alpha=0.5, temperature=3.0):
        super().__init__()
        self.teacher = teacher_model
        self.student = student_model
        self.alpha = alpha
        self.temperature = temperature
        self.criterion_kd = nn.KLDivLoss(reduction="batchmean")
        self.criterion_ce = nn.CrossEntropyLoss()

    def forward(self, x):
        return self.student(x)

    def training_step(self, batch, batch_idx):
        images, labels = batch

        # Teacher predictions
        with torch.no_grad():
            teacher_logits = self.teacher.model(images)  # Assuming 'model' attribute in Patchcore holds the feature extractor

        # Student predictions
        student_logits = self.student(images)

        # Knowledge Distillation Loss
        kd_loss = self.criterion_kd(
            nn.functional.log_softmax(student_logits / self.temperature, dim=1),
            nn.functional.softmax(teacher_logits / self.temperature, dim=1)
        ) * (self.temperature ** 2)

        # Cross-Entropy Loss
        ce_loss = self.criterion_ce(student_logits, labels)

        # Combined Loss
        loss = self.alpha * kd_loss + (1 - self.alpha) * ce_loss

        self.log('train_loss', loss)
        return loss

    def configure_optimizers(self):
        optimizer = optim.Adam(self.student.parameters(), lr=0.001)
        return optimizer

def main():
    # Initialize teacher model (Patchcore)
    teacher_model = Patchcore(
        backbone="wide_resnet50_2",
        layers=["layer2", "layer3"],
        pre_trained=True,
        coreset_sampling_ratio=0.1,
        num_neighbors=9,
    )
    teacher_model.eval()  # Set to evaluation mode

    # Initialize student model
    student_model = StudentModel()

    # Create the distillation model
    distillation_model = DistillationModel(teacher_model, student_model)

    # Data preparation
    transform = Compose([
        Resize((224, 224)),
        ToTensor(),
        Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])

    # Assume you have a dataset in the 'data' directory
    train_dataset = ImageFolder(root='/content/drive/MyDrive/patch_core/Project_Phase_2/ITD/type1cam1', transform=transform)
    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=4)

    # Create the distillation model
    distillation_model = DistillationModel(teacher_model, student_model)

    # Training
    trainer = pl.Trainer(max_epochs=1)
    trainer.fit(distillation_model, train_loader)

    # Save the distilled student model
    torch.save(student_model.state_dict(), 'distilled_resnet18.pth')

if __name__ == "__main__":
    main()

INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.callbacks.model_summary:
  | Name         | Type             | Params | Mode 
----------------------------------------------------------
0 | teacher      | Patchcore        | 24.9 M | eval 
1 | student      | StudentModel     | 11.7 M | train
2 | criterion_kd | KLDivLoss        | 0      | train
3 | criterion_ce | CrossEntropyLoss | 0      | train
----------------------------------------------------------
36.6 M    Trainable params
0         Non-trainable params
36.6 M    Total params
146.208   Total estimated model params size (MB)
71        Modules in train mode
179       Modules in eval mode


Training: |          | 0/? [00:00<?, ?it/s]

IndexError: Dimension out of range (expected to be in range of [-1, 0], but got -2)

In [30]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models
from torch.utils.data import DataLoader
import pytorch_lightning as pl
from torchvision.transforms import Compose, Resize, ToTensor, Normalize
from torchvision.datasets import ImageFolder

from anomalib.models import Patchcore

class StudentModel(nn.Module):
    def __init__(self, feature_dim):
        super().__init__()
        self.model = models.resnet18(pretrained=True)
        num_ftrs = self.model.fc.in_features
        self.model.fc = nn.Identity()  # Remove the final FC layer
        self.fc = nn.Linear(num_ftrs, feature_dim)  # Add a new FC layer to match Patchcore's feature dimension

    def forward(self, x):
        features = self.model(x)
        return self.fc(features)

class DistillationModel(pl.LightningModule):
    def __init__(self, teacher_model, student_model, alpha=0.5):
        super().__init__()
        self.teacher = teacher_model
        self.student = student_model
        self.alpha = alpha
        self.criterion_mse = nn.MSELoss()

    def forward(self, x):
        return self.student(x)

    def training_step(self, batch, batch_idx):
        images, _ = batch

        # Teacher predictions
        with torch.no_grad():
            teacher_features = self.teacher.embed(images)

        # Student predictions
        student_features = self.student(images)

        # Feature matching loss
        loss = self.criterion_mse(student_features, teacher_features)

        self.log('train_loss', loss)
        return loss

    def configure_optimizers(self):
        optimizer = optim.Adam(self.student.parameters(), lr=0.001)
        scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=30, gamma=0.1)
        return [optimizer], [scheduler]

def main():
    # Initialize teacher model (Patchcore)
    teacher_model = Patchcore(
        backbone="wide_resnet50_2",
        layers=["layer2", "layer3"],
        pre_trained=True,
        coreset_sampling_ratio=0.1,
        num_neighbors=9,
    )
    teacher_model.eval()

    # Determine the feature dimension of the teacher model
    with torch.no_grad():
        dummy_input = torch.randn(1, 3, 224, 224)
        teacher_features = teacher_model.embed(dummy_input)
        feature_dim = teacher_features.shape[1]

    # Initialize student model
    student_model = StudentModel(feature_dim)

    # Data preparation
    transform = Compose([
        Resize((224, 224)),
        ToTensor(),
        Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])

    train_dataset = ImageFolder(root='/content/drive/MyDrive/patch_core/Project_Phase_2/ITD/type1cam1', transform=transform)
    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=4)

    # Create the distillation model
    distillation_model = DistillationModel(teacher_model, student_model)

    # Training
    trainer = pl.Trainer(max_epochs=100, accelerator='gpu', devices=1)
    trainer.fit(distillation_model, train_loader)

    # Save the distilled student model
    torch.save(student_model.state_dict(), 'distilled_resnet18.pth')

if __name__ == "__main__":
    main()

AttributeError: 'Patchcore' object has no attribute 'embed'

In [7]:
import torch
# from anomalib.models import Patchcore
from lightning_model import Patchcore

# Initialize the model
teacher_model = Patchcore(
    backbone="wide_resnet50_2",
    layers=["layer2", "layer3"],
    pre_trained=True,
    coreset_sampling_ratio=0.1,
    num_neighbors=9,
)

# Print the model structure
print(teacher_model)

# Try to find methods for feature extraction
print(dir(teacher_model))

# Attempt to use the model for prediction
dummy_input = torch.randn(1, 3, 224, 224)
output = teacher_model(dummy_input)
print("Output:", output,"Outputsize:" ,output.shape)

ModuleNotFoundError: No module named 'lightning_model'

In [6]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models
from torch.utils.data import DataLoader
import pytorch_lightning as pl
from torchvision.transforms import Compose, Resize, ToTensor, Normalize
from torchvision.datasets import ImageFolder

from anomalib.models import Patchcore

class StudentModel(nn.Module):
    def __init__(self, output_dim):
        super().__init__()
        self.model = models.resnet18(pretrained=True)
        num_ftrs = self.model.fc.in_features
        self.model.fc = nn.Identity()  # Remove the final FC layer
        self.fc = nn.Linear(num_ftrs, output_dim)  # Add a new FC layer to match Patchcore's output dimension

    def forward(self, x):
        features = self.model(x)
        return self.fc(features)

class DistillationModel(pl.LightningModule):
    def __init__(self, teacher_model, student_model):
        super().__init__()
        self.teacher = teacher_model
        self.student = student_model
        self.criterion_mse = nn.MSELoss()

    def forward(self, x):
        return self.student(x)

    def training_step(self, batch, batch_idx):
        images, _ = batch

        # Teacher predictions
        with torch.no_grad():
            teacher_features = self.teacher(images)

        # Student predictions
        student_features = self.student(images)

        # Feature matching loss
        loss = self.criterion_mse(student_features, teacher_features)

        self.log('train_loss', loss)
        return loss

    def configure_optimizers(self):
        optimizer = optim.Adam(self.student.parameters(), lr=0.001)
        scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=30, gamma=0.1)
        return [optimizer], [scheduler]

def main():
    # Initialize teacher model (Patchcore)
    teacher_model = Patchcore(
        backbone="wide_resnet50_2",
        layers=["layer2", "layer3"],
        pre_trained=True,
        coreset_sampling_ratio=0.1,
        num_neighbors=9,
    )
    teacher_model.eval()

    # Determine the output dimension of the teacher model
    with torch.no_grad():
        dummy_input = torch.randn(1, 3, 224, 224)
        #teacher_output = teacher_model(dummy_input)
        # output_dim = teacher_output.shape[1]
        output_dim = 1536

    # Initialize student model
    student_model = StudentModel(output_dim)

    # Data preparation
    transform = Compose([
        Resize((224, 224)),
        ToTensor(),
        Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])

    train_dataset = ImageFolder(root='/content/drive/MyDrive/patch_core/Project_Phase_2/ITD/type1cam1', transform=transform)
    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=4)

    # Create the distillation model
    distillation_model = DistillationModel(teacher_model, student_model)

    # Training
    trainer = pl.Trainer(max_epochs=100, accelerator='gpu', devices=1)
    trainer.fit(distillation_model, train_loader)

    # Save the distilled student model
    torch.save(student_model.state_dict(), 'distilled_resnet18.pth')

if __name__ == "__main__":
    main()

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 209MB/s]
/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current 

Training: |          | 0/? [00:00<?, ?it/s]

IndexError: Dimension out of range (expected to be in range of [-1, 0], but got -2)